# Setup infrastructure

In [1]:
from azureml.core import Workspace, Experiment

ws = Workspace.get(name="azure-ml-engineer")
exp = Experiment(workspace=ws, name="udacity-project")



print('Workspace name: ' + ws.name, 
      'Azure region: ' + ws.location, 
      #'Subscription id: ' + ws.subscription_id, 
      'Resource group: ' + ws.resource_group, sep = '\n')

run = exp.start_logging()

Workspace name: azure-ml-engineer
Azure region: northeurope
Resource group: azure-ml-engineer-resource


In [2]:
from azureml.core.compute import ComputeTarget, AmlCompute
from azureml.core.compute_target import ComputeTargetException
# TODO: Create compute cluster
# Use vm_size = "Standard_D2_V2" in your provisioning configuration.
# max_nodes should be no greater than 4.

### YOUR CODE HERE ###
cpu_cluster_name = "vm-azureml-experiment"
try:
    cpu_cluster = ComputeTarget(workspace=ws, name=cpu_cluster_name)
    print('Found existing cluster, use it.')
except ComputeTargetException:
    compute_config = AmlCompute.provisioning_configuration(vm_size='STANDARD_D2_V2',
                                                           max_nodes=4)
    cpu_cluster = ComputeTarget.create(ws, cpu_cluster_name, compute_config)

cpu_cluster.wait_for_completion(show_output=True)

Found existing cluster, use it.

Running


# Using Hyperdrive for hyperparameter tuning

In [3]:
from azureml.core.environment import Environment
from azureml.train.sklearn import SKLearn
from azureml.train.hyperdrive.run import PrimaryMetricGoal
from azureml.train.hyperdrive.policy import BanditPolicy
from azureml.train.hyperdrive.sampling import RandomParameterSampling
from azureml.train.hyperdrive.runconfig import HyperDriveConfig
from azureml.train.hyperdrive.parameter_expressions import uniform
from azureml.train.hyperdrive import choice, PrimaryMetricGoal
from azureml.core.script_run_config import ScriptRunConfig
from azureml.train.estimator import Estimator
import os

# Specify parameter sampler
### YOUR CODE HERE ###
params_sampling = RandomParameterSampling({ 
    "C": choice([0.1, 0.5, 1]), 
    "max_iter": choice([100, 200, 300, 400, 500]), 
}) 


# Specify a Policy
policy = BanditPolicy(evaluation_interval=100, slack_factor=0.2)

if "training" not in os.listdir():
    os.mkdir("./training")

# Create a SKLearn estimator for use with train.py
### YOUR CODE HERE ###
# sklearn class is deprecated. Use ScriptRunConfig instead. 
run_config = ScriptRunConfig(
    source_directory="./", 
    script="train.py", 
    arguments=None, 
    compute_target=cpu_cluster_name, 
    environment=Environment.get(workspace=ws, name='AzureML-Scikit-learn-0.20.3'), 
)


# Create a HyperDriveConfig using the estimator, hyperparameter sampler, and policy.
hyperdrive_config = HyperDriveConfig(
    hyperparameter_sampling=params_sampling, 
    primary_metric_name="Accuracy", 
    primary_metric_goal=PrimaryMetricGoal.MAXIMIZE, 
    max_total_runs=100, 
    policy=policy, 
    run_config=run_config, 
)

In [4]:
# Submit your hyperdrive run to the experiment and show run details with the widget.
### YOUR CODE HERE ###
exp = Experiment(ws, "azureml-project1-hypertuning")

In [93]:
run = exp.submit(hyperdrive_config)

In [5]:
run.get_status()

'Running'

## Get the best run and save the model from that run

In [3]:
from azureml.train.hyperdrive.run import HyperDriveRun

The `run_id` corresponding to the run with the highest accuracy was identified using the azureml studio interface. 

In [4]:
remote_run = HyperDriveRun(
    experiment=Experiment(ws, "azureml-project1-hypertuning"), 
    run_id="HD_c9af2ef4-6287-4985-b0d5-fbeee818e522"
)

best_run = remote_run.get_best_run_by_primary_metric()

parameter_values = best_run.get_details()['runDefinition']['arguments']
best_parameters = dict(zip(parameter_values[::2], parameter_values[1::2]))
print(best_parameters)

{'--C': '1', '--max_iter': '300'}


The `train.py` script was modified so that it persists the fitted model if `model_dir` and `model_name` are provided. 

In [65]:
!python train.py --C 1 --max_iter 300 --model_dir ./ --model_name best_model

Failure while loading azureml_run_type_providers. Failed to load entrypoint hyperdrive = azureml.train.hyperdrive:HyperDriveRun._from_run_dto with exception (pyOpenSSL 20.0.0 (/anaconda/envs/azureml_py36/lib/python3.6/site-packages), Requirement.parse('pyopenssl<20.0.0'), {'azureml-core'}).
Failure while loading azureml_run_type_providers. Failed to load entrypoint automl = azureml.train.automl.run:AutoMLRun._from_run_dto with exception (pyOpenSSL 20.0.0 (/anaconda/envs/azureml_py36/lib/python3.6/site-packages), Requirement.parse('pyopenssl<20.0.0'), {'azureml-core'}).
Failure while loading azureml_run_type_providers. Failed to load entrypoint azureml.PipelineRun = azureml.pipeline.core.run:PipelineRun._from_dto with exception (pyOpenSSL 20.0.0 (/anaconda/envs/azureml_py36/lib/python3.6/site-packages), Requirement.parse('pyopenssl<20.0.0'), {'azureml-core'}).
Failure while loading azureml_run_type_providers. Failed to load entrypoint azureml.ReusedStepRun = azureml.pipeline.core.run:St

## Visualization

Visualize the run in details using the `azureml.widgets.RunDetails` tool: 

In [5]:
from azureml.widgets import RunDetails
RunDetails(remote_run).show()

_HyperDriveWidget(widget_settings={'childWidgetDisplay': 'popup', 'send_telemetry': False, 'log_level': 'INFO'…

# AutoML

In [6]:
from azureml.data.dataset_factory import TabularDatasetFactory

# Create TabularDataset using TabularDatasetFactory
# Data is available at: 
# "https://automlsamplenotebookdata.blob.core.windows.net/automl-sample-notebook-data/bankmarketing_train.csv"

### YOUR CODE HERE ###
file_path = "https://automlsamplenotebookdata.blob.core.windows.net/automl-sample-notebook-data/bankmarketing_train.csv" 
dataset = TabularDatasetFactory.from_delimited_files(file_path)

## Prepare the data for the AutoML run

For a comparison with the hyperparameter tunning run, we also clean the data using the `clean_data` function. 

In [7]:
from train import clean_data

# Use the clean_data function to clean your data.
x, y = clean_data(dataset)

from sklearn.model_selection import train_test_split
x_train, x_test, y_train, y_test = train_test_split(
    x, y, test_size=0.33, random_state=1234
)

training_data = x_train.copy()
training_data["y"] = y_train

validation_data = x_test.copy()
validation_data["y"] = y_test

Now, we register the training data and validation as datasets to be used later by the AutoML run. 

In [14]:
from azureml.core import Datastore

datastore = Datastore.get(ws, 'workspaceblobstore')

Dataset.Tabular.register_pandas_dataframe(
    dataframe=training_data, 
    name='bank_market_automl_training_set', 
    target=datastore
)

Dataset.Tabular.register_pandas_dataframe(
    dataframe=validation_data, 
    name='bank_market_automl_validation_set', 
    target=datastore
)

In [90]:
from azureml.core import Dataset
from azureml.train.automl import AutoMLConfig

# Set parameters for AutoMLConfig
# NOTE: DO NOT CHANGE THE experiment_timeout_minutes PARAMETER OR YOUR INSTANCE WILL TIME OUT.
# If you wish to run the experiment longer, you will need to run this notebook in your own
# Azure tenant, which will incur personal costs.
automl_config = AutoMLConfig(
    experiment_timeout_minutes=30,
    compute_target=cpu_cluster_name,
    task="classification",
    primary_metric="accuracy",
    training_data=Dataset.get_by_name(ws, 'bank_market_automl_training_set'),
    validation_data=Dataset.get_by_name(ws, 'bank_market_automl_validation_set'),
    label_column_name="y")

automl_config_crossval = AutoMLConfig(
    experiment_timeout_minutes=30,
    compute_target=cpu_cluster_name,
    task="classification",
    primary_metric="accuracy",
    training_data=Dataset.get_by_name(ws, 'bank_market_automl_training_set'),
    n_cross_validations=5,
    label_column_name="y")

In [91]:
# Submit your automl run
run = Experiment(ws, "azureml-autoML-crossval")
### YOUR CODE HERE ###
run.submit(automl_config_crossval)

Running on remote.


Experiment,Id,Type,Status,Details Page,Docs Page
azureml-autoML-crossval,AutoML_58841890-3a97-4ccb-aee3-78bbe455e810,automl,NotStarted,Link to Azure Machine Learning studio,Link to Documentation


The `run_id` corresponding to the run that returned the best model, based on accuracy, was identified using azureml studio. 

In [8]:
# Retrieve and save your best automl model.

### YOUR CODE HERE ###
from azureml.train.automl.run import AutoMLRun
remote_run = AutoMLRun(experiment=ws.experiments['azureml-autoML-2'], run_id='AutoML_cbb0382a-bc54-4e5e-9280-bff27fe15ee2')
remote_run

Experiment,Id,Type,Status,Details Page,Docs Page
azureml-autoML-2,AutoML_cbb0382a-bc54-4e5e-9280-bff27fe15ee2,automl,Completed,Link to Azure Machine Learning studio,Link to Documentation


In [9]:
best_run, fitted_model = remote_run.get_output()
print(fitted_model)

Pipeline(memory=None,
         steps=[('datatransformer',
                 DataTransformer(enable_dnn=None, enable_feature_sweeping=None,
                                 feature_sweeping_config=None,
                                 feature_sweeping_timeout=None,
                                 featurization_config=None, force_text_dnn=None,
                                 is_cross_validation=None,
                                 is_onnx_compatible=None, logger=None,
                                 observer=None, task=None, working_dir=None)),
                ('prefittedsoftvotingclassifier',...
                                                                                                multi_class='ovr',
                                                                                                n_jobs=1,
                                                                                                penalty='l1',
                                                             

In [10]:
classifier = fitted_model.named_steps['prefittedsoftvotingclassifier']

In [11]:
weights = classifier.weights
idx_max_weight = weights.index(max(weights))
classifier.estimators[idx_max_weight]

('1',
 Pipeline(memory=None,
          steps=[('maxabsscaler', MaxAbsScaler(copy=True)),
                 ('xgboostclassifier',
                  XGBoostClassifier(base_score=0.5, booster='gbtree',
                                    colsample_bylevel=1, colsample_bynode=1,
                                    colsample_bytree=1, gamma=0,
                                    learning_rate=0.1, max_delta_step=0,
                                    max_depth=3, min_child_weight=1, missing=nan,
                                    n_estimators=100, n_jobs=1, nthread=None,
                                    objective='binary:logistic', random_state=0,
                                    reg_alpha=0, reg_lambda=1,
                                    scale_pos_weight=1, seed=None, silent=None,
                                    subsample=1, tree_method='auto', verbose=-10,
                                    verbosity=0))],
          verbose=False))

Best run using `n_cross_validations`:

In [12]:
from azureml.train.automl.run import AutoMLRun
remote_run_crossval = AutoMLRun(experiment=ws.experiments['azureml-autoML-crossval'], run_id='AutoML_58841890-3a97-4ccb-aee3-78bbe455e810')
best_run, fitted_model = remote_run_crossval.get_output()

In [13]:
best_run

Experiment,Id,Type,Status,Details Page,Docs Page
azureml-autoML-crossval,AutoML_58841890-3a97-4ccb-aee3-78bbe455e810_29,azureml.scriptrun,Completed,Link to Azure Machine Learning studio,Link to Documentation


In [14]:
classifier = fitted_model.named_steps['prefittedsoftvotingclassifier']

In [15]:
weights = classifier.weights
idx_max_weight = weights.index(max(weights))
classifier.estimators[idx_max_weight]

('1',
 Pipeline(memory=None,
          steps=[('maxabsscaler', MaxAbsScaler(copy=True)),
                 ('xgboostclassifier',
                  XGBoostClassifier(base_score=0.5, booster='gbtree',
                                    colsample_bylevel=1, colsample_bynode=1,
                                    colsample_bytree=1, gamma=0,
                                    learning_rate=0.1, max_delta_step=0,
                                    max_depth=3, min_child_weight=1, missing=nan,
                                    n_estimators=100, n_jobs=1, nthread=None,
                                    objective='binary:logistic', random_state=0,
                                    reg_alpha=0, reg_lambda=1,
                                    scale_pos_weight=1, seed=None, silent=None,
                                    subsample=1, tree_method='auto', verbose=-10,
                                    verbosity=0))],
          verbose=False))

# Delete cluster

In [16]:
cpu_cluster.delete()